# Experiment 007: Tessellation Approach for Large N

Implement the tessellation approach from egortrushin kernel for large N values.
This generates solutions from scratch with a different structure that may escape the local optimum.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from decimal import Decimal, getcontext
import random
import copy
import time
import math
import os
import json

getcontext().prec = 30

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

class ChristmasTree:
    def __init__(self, center_x=0, center_y=0, angle=0):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self._update_polygon()
    
    def _update_polygon(self):
        # Create base polygon
        base = Polygon(zip(TX, TY))
        # Rotate around origin
        rotated = affinity.rotate(base, float(self.angle), origin=(0, 0))
        # Translate to center
        self.polygon = affinity.translate(rotated, float(self.center_x), float(self.center_y))
    
    def get_params(self):
        return self.center_x, self.center_y, self.angle
    
    def set_params(self, x, y, angle):
        self.center_x = Decimal(str(x))
        self.center_y = Decimal(str(y))
        self.angle = Decimal(str(angle))
        self._update_polygon()

print("ChristmasTree class defined")

ChristmasTree class defined


In [2]:
def has_collision(trees):
    """Check if any trees overlap"""
    for i in range(len(trees)):
        for j in range(i+1, len(trees)):
            if trees[i].polygon.intersects(trees[j].polygon) and not trees[i].polygon.touches(trees[j].polygon):
                inter = trees[i].polygon.intersection(trees[j].polygon)
                if inter.area > 1e-10:
                    return True
    return False

def calculate_score_for_trees(trees):
    """Calculate bounding box side for a list of trees"""
    all_coords = []
    for t in trees:
        coords = np.array(t.polygon.exterior.coords)
        all_coords.append(coords)
    all_coords = np.vstack(all_coords)
    x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
    y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
    return max(x_range, y_range)

print("Helper functions defined")

Helper functions defined


In [3]:
def translate_trees(base_trees, lengthx, lengthy, nt):
    """Translate base trees to create a grid pattern"""
    trees = []
    for tree in base_trees:
        for x in range(nt[0]):
            for y in range(nt[1]):
                new_tree = ChristmasTree(
                    center_x=float(tree.center_x) + x * lengthx,
                    center_y=float(tree.center_y) + y * lengthy,
                    angle=float(tree.angle)
                )
                trees.append(new_tree)
    return trees

def get_optimal_lengths(base_trees, nt, delta=0.001):
    """Find optimal translation lengths that minimize bounding box without collisions"""
    # Start with bounding box of base trees
    all_coords = []
    for t in base_trees:
        coords = np.array(t.polygon.exterior.coords)
        all_coords.append(coords)
    all_coords = np.vstack(all_coords)
    
    # Initial length estimate
    x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
    y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
    length = max(x_range, y_range)
    
    lengthx = length
    lengthy = length
    
    # Shrink lengthx until collision
    while True:
        trees = translate_trees(base_trees, lengthx - delta, lengthy, nt)
        if has_collision(trees):
            break
        lengthx -= delta
    
    # Shrink lengthy until collision
    while True:
        trees = translate_trees(base_trees, lengthx, lengthy - delta, nt)
        if has_collision(trees):
            break
        lengthy -= delta
    
    return lengthx, lengthy

print("Translation functions defined")

Translation functions defined


In [4]:
# Test with the initial trees from egortrushin kernel
initial_trees = [
    ChristmasTree(-2.93069232, -4.24856960, 67),
    ChristmasTree(-3.92971914, -4.16631769, 250.00)
]

print("Initial trees:")
for i, t in enumerate(initial_trees):
    print(f"  Tree {i}: x={float(t.center_x):.4f}, y={float(t.center_y):.4f}, angle={float(t.angle):.2f}")

# Test for N=72 (nt=[4,9], 2*4*9=72)
nt = [4, 9]
lengthx, lengthy = get_optimal_lengths(initial_trees, nt)
print(f"\nFor N=72 (nt={nt}):")
print(f"  lengthx={lengthx:.4f}, lengthy={lengthy:.4f}")

trees_72 = translate_trees(initial_trees, lengthx, lengthy, nt)
print(f"  Generated {len(trees_72)} trees")
print(f"  Has collision: {has_collision(trees_72)}")
side_72 = calculate_score_for_trees(trees_72)
print(f"  Bounding box side: {side_72:.6f}")
print(f"  Score contribution: {side_72**2 / 72:.6f}")

Initial trees:
  Tree 0: x=-2.9307, y=-4.2486, angle=67.00
  Tree 1: x=-3.9297, y=-4.1663, angle=250.00



For N=72 (nt=[4, 9]):
  lengthx=1.2560, lengthy=0.5670
  Generated 72 trees
  Has collision: False
  Bounding box side: 5.269502
  Score contribution: 0.385662


In [5]:
# Load current best submission to compare
df_best = pd.read_csv('/home/nonroot/snapshots/santa-2025/21164519357/code/exploration/santa-2025.csv')

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def get_current_score_for_n(df, n):
    """Get current score for a specific N"""
    prefix = f"{n:03d}_"
    n_trees = df[df['id'].str.startswith(prefix)]
    if len(n_trees) != n:
        return None
    
    all_coords = []
    for _, row in n_trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        tree = ChristmasTree(x, y, deg)
        coords = np.array(tree.polygon.exterior.coords)
        all_coords.append(coords)
    
    all_coords = np.vstack(all_coords)
    x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
    y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
    return max(x_range, y_range)

# Compare tessellation vs current for N=72
current_side_72 = get_current_score_for_n(df_best, 72)
print(f"N=72 comparison:")
print(f"  Current: side={current_side_72:.6f}, score={current_side_72**2/72:.6f}")
print(f"  Tessellation: side={side_72:.6f}, score={side_72**2/72:.6f}")
print(f"  Improvement: {current_side_72**2/72 - side_72**2/72:.6f}")

N=72 comparison:
  Current: side=5.009617, score=0.348559
  Tessellation: side=5.269502, score=0.385662
  Improvement: -0.037103


In [6]:
# Try tessellation for multiple N values
n_to_nt = {
    72: [4, 9],
    100: [5, 10],
    110: [5, 11],
    144: [6, 12],
    156: [6, 13],
    196: [7, 14],
    200: [7, 15]  # Will generate 210, take first 200
}

results = {}
for n, nt in n_to_nt.items():
    print(f"\nProcessing N={n} with nt={nt}...")
    
    # Get optimal lengths
    lengthx, lengthy = get_optimal_lengths(initial_trees, nt)
    
    # Generate trees
    trees = translate_trees(initial_trees, lengthx, lengthy, nt)
    
    # For N=200, take first 200 trees
    if n == 200:
        trees = trees[:200]
    
    # Check for collisions
    if has_collision(trees):
        print(f"  WARNING: Has collisions!")
        continue
    
    # Calculate score
    side = calculate_score_for_trees(trees)
    score = side**2 / n
    
    # Compare with current
    current_side = get_current_score_for_n(df_best, n)
    current_score = current_side**2 / n
    
    improvement = current_score - score
    results[n] = {
        'side': side,
        'score': score,
        'current_side': current_side,
        'current_score': current_score,
        'improvement': improvement,
        'trees': trees
    }
    
    print(f"  Current: side={current_side:.6f}, score={current_score:.6f}")
    print(f"  Tessellation: side={side:.6f}, score={score:.6f}")
    print(f"  Improvement: {improvement:.6f} ({'BETTER' if improvement > 0 else 'WORSE'})")


Processing N=72 with nt=[4, 9]...


  Current: side=5.009617, score=0.348559
  Tessellation: side=5.269502, score=0.385662
  Improvement: -0.037103 (WORSE)

Processing N=100 with nt=[5, 10]...


  Current: side=5.878187, score=0.345531
  Tessellation: side=6.450113, score=0.416040
  Improvement: -0.070509 (WORSE)

Processing N=110 with nt=[5, 11]...


  Current: side=6.093960, score=0.337603
  Tessellation: side=6.450113, score=0.378218
  Improvement: -0.040615 (WORSE)

Processing N=144 with nt=[6, 12]...


  Current: side=7.020525, score=0.342276
  Tessellation: side=7.706136, score=0.412393
  Improvement: -0.070116 (WORSE)

Processing N=156 with nt=[6, 13]...


  Current: side=7.174802, score=0.329986
  Tessellation: side=7.706136, score=0.380670
  Improvement: -0.050684 (WORSE)

Processing N=196 with nt=[7, 14]...


  Current: side=8.082085, score=0.333266
  Tessellation: side=8.962159, score=0.409797
  Improvement: -0.076532 (WORSE)

Processing N=200 with nt=[7, 15]...


  Current: side=8.216618, score=0.337564
  Tessellation: side=8.962159, score=0.401601
  Improvement: -0.064037 (WORSE)


In [7]:
# Summary of results
print("\n" + "="*60)
print("SUMMARY")
print("="*60)

total_improvement = 0
for n in sorted(results.keys()):
    r = results[n]
    status = "BETTER" if r['improvement'] > 0 else "WORSE"
    print(f"N={n:3d}: improvement={r['improvement']:+.6f} ({status})")
    if r['improvement'] > 0:
        total_improvement += r['improvement']

print(f"\nTotal potential improvement: {total_improvement:.6f}")


SUMMARY
N= 72: improvement=-0.037103 (WORSE)
N=100: improvement=-0.070509 (WORSE)
N=110: improvement=-0.040615 (WORSE)
N=144: improvement=-0.070116 (WORSE)
N=156: improvement=-0.050684 (WORSE)
N=196: improvement=-0.076532 (WORSE)
N=200: improvement=-0.064037 (WORSE)

Total potential improvement: 0.000000


In [ ]:
# The tessellation approach doesn't beat the current solution
# This is expected - the current solution is already highly optimized
# Let's save the current best submission

import shutil
shutil.copy('/home/nonroot/snapshots/santa-2025/21164519357/code/exploration/santa-2025.csv',
            '/home/submission/submission.csv')

# Calculate total score
total_score = 0
for n in range(1, 201):
    side = get_current_score_for_n(df_best, n)
    if side:
        total_score += side**2 / n

print(f"Total score: {total_score:.6f}")

# Save metrics
os.makedirs('/home/code/experiments/007_tessellation', exist_ok=True)
metrics = {'cv_score': total_score}
with open('/home/code/experiments/007_tessellation/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Saved metrics: {metrics}")